In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from hidden_others.ch06.rnnlm import Rnnlm
from hidden_others.ch06.better_rnnlm import BetterRnnlm
from common.functions import softmax

In [3]:
class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids=[start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())

            sampled = np.random.choice(len(p), size=1, p=p)

            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))


        return word_ids

In [4]:
# 実際に文書生成してみる
from hidden_others.dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

start_word = 'you'
start_id = word_to_id[start_word]
model = RnnlmGen()
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文書生成
word_ids = model.generate(start_id, skip_ids=skip_ids)
txt = ' '.join([id_to_word[id] for id in word_ids])
txt = txt.replace('<eos>', '.\n')
print(txt)


you trespass wis. cat communities poured cowboys sony rtc materials weighing fla. arkansas remains including refusing angry corners shipped burnham employ difficult lobbying organic congressman foothills consumed nearby doldrums heritage inventories bullet justifies studio burden most protest burns approve banponce inappropriate preparation slowly shaky smaller critical led knowledge electoral lucrative caution petco detailed south jeff aoun occurs per gaf stick appropriate programming scenario seven-year fairfield deliberately carbon grants investor initiated resistance requires homeowners ohbayashi mural cover stadium towers annuities bikers camera statute conflicts chip identifying planners pegged cheaper cities relationship neat tables aviation convenient bullets gathering figures o. futures attracting


In [5]:
# 学習済みモデルで生成
# 実際に文書生成してみる
from hidden_others.dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

start_word = 'you'
start_id = word_to_id[start_word]
model = RnnlmGen()
model.load_params(file_name='Rnnlm.pkl')
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文書生成

word_ids = model.generate(start_id, skip_ids=skip_ids)
txt = ' '.join([id_to_word[id] for id in word_ids])
txt = txt.replace('<eos>', '.\n')
print(txt)

you also do n't day .
 in earnings .
 leaping into those who gauge bound injection of government loans from the federal court .
 joseph 's of counting improving it in a short significantly financial move and that it will take some base corn .
 using charts will be able to form a tumultuous much of the total of infiniti daily shares as well as well unload cleaning up about thursday copies of an rebuild slide .
 while they can get a heavy treasurys of rival naked growth during the pop however is almost signs of people explaining time out


In [6]:
# lstmモデルでもやってみる
from hidden_others.ch06.better_rnnlm import BetterRnnlm

class betterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids=[start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())

            sampled = np.random.choice(len(p), size=1, p=p)

            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))


        return word_ids



corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

start_word = 'you'
start_id = word_to_id[start_word]
model = betterRnnlmGen()
model.load_params(file_name='hidden_others/BetterRnnlm.pkl')
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文書生成

word_ids = model.generate(start_id, skip_ids=skip_ids)
txt = ' '.join([id_to_word[id] for id in word_ids])
txt = txt.replace('<eos>', '.\n')
print(txt)

you project the store in a red from the thinking of me last year .
 he unit 's most lucrative worker learned it goupil never attempted because his hepatitis 's business was made .
 the team 's team change a side note we morning under the port of toronto .
 by beginning a four-game area may seek left from the new project in the southwest .
 this is a theater for a while and look like it or all the show and huge stuff at us like is .
 many speak watch it in a country like that makes panels


In [7]:
# the meaning of lileで始まる文章
model.reset_state()

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

# predictで最後の文字以外の文字を順番に与えて情報を持たせる
for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x)

# 最後の文字から文章生成スタート
word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids # 最初に与えたワードを先頭に与える
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)

--------------------------------------------------
the meaning of life is n't really consistent but added that bells swing with weakest and boomers are well off.
 in business news on integrated 's tumbled the ski system 's health operations were turned steadily.
 in a number of occasions and said they are indeed very favorite.
 the conviction of a first reference to the field 's punitive damages among the installations was 's merit.
 a lot of fraud has and talked to people.
 we alike are trying to condemn the project.
 for instance that hope.
 what sent my pictures to be sure i did n't


In [9]:
# 足し算データセット

from hidden_others.dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)


In [23]:
# Encoder 
from common.time_layers import *

class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        enbed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, H*4) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, H*4) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(H*4).astype('f')

        self.embed = TimeEmbedding(enbed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False) # １ブロックでdecoderに渡すので保持する理由がない

        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None


    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :]


    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh # 最後の時系列以外は廃棄 